# Titanic Dataset

In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
import seaborn as sns
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


C:\Users\akank\anaconda3\Lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['clf']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [54]:
titanic_text=pd.read_csv('Titanic.txt', 
                    sep=',', quotechar='"', encoding='ISO 8859-15')
titanic_text.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,ParCh,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Data Exploration and Preparation

Learn about data
1. Are there missing data?
2. Is it categorical? if not, min , max, avg values? if yes, what are the categories?
3. distribution of variables
4. Duplicate entry

### This will helps us to give tha idea about missing data 

### Handling Missing data

Strategies
<li> Drop any observation with missing age
<li> Set NaN as 0 - in the case  continous / categorical variables, create some new category for Nan?
<li> use average value..
<li> create Ml algo to predict missing values(ex-linear regression)

### categorical variable

In [55]:
titanic_text =titanic_text.drop([918,920]) #Two entries having name entered in place of class.
                    #Now we have two options either to drop or search for the dataset and fill the data.

In [56]:
titanic_text.Survived.value_counts() # 0- Didn't survive 1- Survived

Survived
0.0    584
1.0    361
Name: count, dtype: int64

### Data Representation and Transformation

### Droping some of the columns which many not contribute much to our machine learning model such as Name, Ticket, Cabin etc

In [57]:
titanic_text['Pclass']=titanic_text['Pclass'].astype(str).astype(int)

In [58]:
titanic_text.info()

<class 'pandas.core.frame.DataFrame'>
Index: 948 entries, 0 to 949
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  948 non-null    int64  
 1   Survived     945 non-null    float64
 2   Pclass       948 non-null    int32  
 3   Name         948 non-null    object 
 4   Sex          948 non-null    object 
 5   Age          763 non-null    float64
 6   SibSp        948 non-null    int64  
 7   ParCh        948 non-null    int64  
 8   Ticket       948 non-null    object 
 9   Fare         948 non-null    float64
 10  Cabin        217 non-null    object 
 11  Embarked     946 non-null    object 
dtypes: float64(3), int32(1), int64(3), object(5)
memory usage: 92.6+ KB


In [59]:
cols = ['Name','Ticket','Cabin','PassengerId']
titanic_text_DT= titanic_text
titanic_text_DT = titanic_text_DT.drop(cols,axis=1)
titanic_text_DT.head()

,Survived,Pclass,Sex,Age,SibSp,ParCh,Fare,Embarked
0,0.0,3,male,22.0,1,0,7.2500,S
1,1.0,1,female,38.0,1,0,71.2833,C
2,1.0,3,female,26.0,0,0,7.9250,S
3,1.0,1,female,35.0,1,0,53.1000,S
4,0.0,3,male,35.0,0,0,8.0500,S


### Now we convert the Pclass, Sex, Embarked to columns in pandas and drop them after conversion.

In [60]:
dummies = []
cols = ['Pclass','Sex','Embarked']
for col in cols:
 dummies.append(pd.get_dummies(titanic_text_DT[col]))

In [61]:
titanic_dummies = pd.concat(dummies, axis=1)
titanic_dummies.head()

,1,2,3,female,male,C,Q,S
0,False,False,True,False,True,False,False,True
1,True,False,False,True,False,True,False,False
2,False,False,True,True,False,False,False,True
3,True,False,False,True,False,False,False,True
4,False,False,True,False,True,False,False,True


### Then  we concatenate to the original dataframe columnwise

In [62]:
titanic_text_DT = pd.concat((titanic_text_DT,titanic_dummies),axis=1)

In [63]:
titanic_text_DT.head()

,Survived,Pclass,Sex,Age,SibSp,ParCh,Fare,Embarked,1,2,3,female,male,C,Q,S
0,0.0,3,male,22.0,1,0,7.2500,S,False,False,True,False,True,False,False,True
1,1.0,1,female,38.0,1,0,71.2833,C,True,False,False,True,False,True,False,False
2,1.0,3,female,26.0,0,0,7.9250,S,False,False,True,True,False,False,False,True
3,1.0,1,female,35.0,1,0,53.1000,S,True,False,False,True,False,False,False,True
4,0.0,3,male,35.0,0,0,8.0500,S,False,False,True,False,True,False,False,True


### Now that we converted Pclass, Sex, Embarked values into columns, we drop the redundant same columns from the dataframe

In [64]:
titanic_text_DT = titanic_text_DT.drop(['Pclass','Sex','Embarked'],axis=1)
titanic_text_DT.head()

,Survived,Age,SibSp,ParCh,Fare,1,2,3,female,male,C,Q,S
0,0.0,22.0,1,0,7.2500,False,False,True,False,True,False,False,True
1,1.0,38.0,1,0,71.2833,True,False,False,True,False,True,False,False
2,1.0,26.0,0,0,7.9250,False,False,True,True,False,False,False,True
3,1.0,35.0,1,0,53.1000,True,False,False,True,False,False,False,True
4,0.0,35.0,0,0,8.0500,False,False,True,False,True,False,False,True


In [65]:
titanic_text_DT.info()

<class 'pandas.core.frame.DataFrame'>
Index: 948 entries, 0 to 949
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  945 non-null    float64
 1   Age       763 non-null    float64
 2   SibSp     948 non-null    int64  
 3   ParCh     948 non-null    int64  
 4   Fare      948 non-null    float64
 5   1         948 non-null    bool   
 6   2         948 non-null    bool   
 7   3         948 non-null    bool   
 8   female    948 non-null    bool   
 9   male      948 non-null    bool   
 10  C         948 non-null    bool   
 11  Q         948 non-null    bool   
 12  S         948 non-null    bool   
dtypes: bool(8), float64(3), int64(2)
memory usage: 51.8 KB


### All is good, except Survived which has few missing values. Pandas has a nice interpolate() function that will replace all the missing NaNs to interpolated values.

In [66]:
titanic_text_DT['Survived'] = titanic_text_DT['Survived'].interpolate()

In [67]:
titanic_text_DT.info()

<class 'pandas.core.frame.DataFrame'>
Index: 948 entries, 0 to 949
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  948 non-null    float64
 1   Age       763 non-null    float64
 2   SibSp     948 non-null    int64  
 3   ParCh     948 non-null    int64  
 4   Fare      948 non-null    float64
 5   1         948 non-null    bool   
 6   2         948 non-null    bool   
 7   3         948 non-null    bool   
 8   female    948 non-null    bool   
 9   male      948 non-null    bool   
 10  C         948 non-null    bool   
 11  Q         948 non-null    bool   
 12  S         948 non-null    bool   
dtypes: bool(8), float64(3), int64(2)
memory usage: 51.8 KB


### Now we convert our dataframe from pandas to numpy and we assign input and output

In [68]:
y = titanic_text_DT['Survived'].values #output

In [69]:
titanic_text_DT.head()

,Survived,Age,SibSp,ParCh,Fare,1,2,3,female,male,C,Q,S
0,0.0,22.0,1,0,7.2500,False,False,True,False,True,False,False,True
1,1.0,38.0,1,0,71.2833,True,False,False,True,False,True,False,False
2,1.0,26.0,0,0,7.9250,False,False,True,True,False,False,False,True
3,1.0,35.0,1,0,53.1000,True,False,False,True,False,False,False,True
4,0.0,35.0,0,0,8.0500,False,False,True,False,True,False,False,True


### X has still Survived values in it, which should not be there. So we drop in numpy column which is the 1st column

In [70]:
titanic_text_ml= titanic_text_DT.drop(['Survived'],axis=1)
X = titanic_text_ml.values
titanic_text_ml.head()

,Age,SibSp,ParCh,Fare,1,2,3,female,male,C,Q,S
0,22.0,1,0,7.2500,False,False,True,False,True,False,False,True
1,38.0,1,0,71.2833,True,False,False,True,False,True,False,False
2,26.0,0,0,7.9250,False,False,True,True,False,False,False,True
3,35.0,1,0,53.1000,True,False,False,True,False,False,False,True
4,35.0,0,0,8.0500,False,False,True,False,True,False,False,True


### Now that we are ready with X and y, lets split the dataset for 70% Training and  30% test set using train_test split

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=0)

### Lets start with simple Decision Tree Classifier machine learning algorithm and see how it goes

In [72]:
clf = tree.DecisionTreeClassifier(max_depth=4)
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

0.7649122807017544